In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../common_routines/')
import numpy as np
from relevant_functions import\
    evaluate_model_score_given_predictions,\
    evaluate_model_score,\
    evaluate_neg_model_score,\
    cross_val_score_given_model,\
    fit_pipeline_and_cross_validate, \
    fit_pipeline_and_evaluate_on_validation_set, \
    print_model_stats_from_pipeline, \
    get_validated_transformed_data
from sklearn import linear_model 

In [2]:
complete_train_data = pd.read_csv("../../input/train.csv")
test_data = pd.read_csv("../../input/test.csv")

In [3]:
# Dump alll the dataframes with one hot encoding.
train_data_one_hot = pd.read_csv('../../cleaned_input/train_data_one_hot.csv')
validation_data_one_hot = pd.read_csv('../../cleaned_input/validation_data_one_hot.to_csv')
test_data_one_hot =pd.read_csv('../../cleaned_input/test_data_one_hot.csv')

# Dump the data frames prior to taking the one hot encoding transformation.
# Remember that we had handled the null values at the stage and hence the model
# does not need to worry about the same.
train_data = pd.read_csv('../../cleaned_input/train_data.csv')
validation_data = pd.read_csv('../../cleaned_input/validation_data.csv')


Now, that we have a reliable way to perform cross validation, we combine training and validation data to one data frame and use that for training/ cross validation.

In [4]:
train_validation_data_one_hot = pd.concat([train_data_one_hot, validation_data_one_hot])

In [5]:
train_validation_data = pd.concat([train_data, validation_data])

Make log transforms on the Y variable.

In [6]:
train_validation_data['LogSalePrice'] = train_validation_data['SalePrice'].apply(lambda x : np.log(1.0 + x))
train_validation_data_one_hot['LogSalePrice'] = \
    train_validation_data_one_hot['SalePrice'].apply(lambda x : np.log(1.0 + x))

In [7]:
train_validation_data['LogSalePricePerSqFeet'] = \
    train_validation_data['LogSalePrice'] - train_validation_data['LogGrLivArea']
train_validation_data_one_hot['LogSalePricePerSqFeet'] = \
    train_validation_data_one_hot['LogSalePrice'] - train_validation_data_one_hot['LogGrLivArea']


To avoid, confusion, we will be always using one hot encoded dataframes. In case  we want to use the direct value of a categorical variable( which is not present in the dataframe, we will be adding a column corresponding to that variable and then use the same.

Also, we reset the index to use something that makes sense.

In [8]:
train_validation_data_one_hot.reset_index(drop=True, inplace=True)  
train_validation_data.reset_index(drop=True, inplace=True)

Now, let us copy over relevant functions from old notebook.

In [9]:
def get_rel_X_cols(input_df, validation_df, X_columns, X_column_transform_map):
    rel_X_cols = list()
    for col in X_columns:
        if col in X_column_transform_map.keys():
            rel_col = X_column_transform_map.get(col)(col, input_df, validation_df)
        else:
            rel_col = [col]
        for elem in rel_col:
            rel_X_cols.append(elem)
    return rel_X_cols

In [10]:
def get_trained_model(X, Y):
    my_model = linear_model.LinearRegression()
    my_model.fit(X,Y)

    return my_model

In [11]:
def get_cross_val_output(input_df, 
                         X_columns=['LogGrLivArea'], 
                         X_column_transform_map={}, 
                         Y_column = 'LogSalePricePerSqFeet', 
                         nfolds=5):
    partition_indices = np.array_split(np.arange(len(input_df)), nfolds)
    
    cross_validated_scores = np.zeros(nfolds)
    cross_validated_data = pd.DataFrame(columns=input_df.columns)
    for i in range(nfolds):
        cross_validated_set = input_df[partition_indices[i][0]:partition_indices[i][-1] + 1].copy()
        rel_training_data = pd.DataFrame(columns=input_df.columns, dtype=float)
        for j in range(nfolds):
            if j != i:
                training_set = input_df[partition_indices[j][0]:partition_indices[j][-1] + 1]
                rel_training_data = pd.concat([rel_training_data, training_set])

        rel_X_cols = get_rel_X_cols(rel_training_data, cross_validated_set, X_columns, X_column_transform_map)
        my_model = get_trained_model(rel_training_data[rel_X_cols], 
                                     rel_training_data[[Y_column]].values.ravel())
        newX = cross_validated_set[rel_X_cols]
        newY = cross_validated_set[[Y_column]]
        cross_validated_score = evaluate_model_score(my_model, newX, newY)
        cross_validated_scores[i] = cross_validated_score
        
    return cross_validated_scores

In [12]:
def get_validation_output(input_df, 
                          validation_df, 
                          X_columns=['LogGrLivArea'], 
                          X_column_transform_map={}, 
                          Y_column = 'LogSalePricePerSqFeet', 
                          nfolds=5):
    rel_X_cols = get_rel_X_cols(input_df, validation_df, X_columns, X_column_transform_map)
    my_model = get_trained_model(input_df[rel_X_cols], 
                                 input_df[[Y_column]].values.ravel())

    newX = validation_df[rel_X_cols]
    newY = validation_df[[Y_column]]
    cross_validated_score = evaluate_model_score(my_model, newX, newY)
        
    return cross_validated_score 

In [13]:
def get_test_data_predictions(input_df, 
                              test_df, 
                              X_columns=['LogGrLivArea'], 
                              X_column_transform_map={}, 
                              Y_column = 'LogSalePricePerSqFeet', 
                              nfolds=5):
    rel_X_cols = get_rel_X_cols(input_df, test_df, X_columns, X_column_transform_map)        
    my_model = get_trained_model(input_df[rel_X_cols], 
                                 input_df[[Y_column]].values.ravel())
    
    newX = test_df[rel_X_cols]
    predicitons =  my_model.predict(newX)
        
    return predicitons

Now, we are all set. Let us start playing with this !

In [14]:
def get_MSSubClass_indicator_transformation(group_col, train_data, validation_data):
    train_data['MSSubClass_60_75_120_20'] = train_data['MSSubClass_120'] + \
    train_data['MSSubClass_60'] + train_data['MSSubClass_20'] + train_data['MSSubClass_75'] 
    validation_data['MSSubClass_60_75_120_20'] = validation_data['MSSubClass_120'] + \
    validation_data['MSSubClass_60'] + validation_data['MSSubClass_20'] + validation_data['MSSubClass_75']
    return ['MSSubClass_60_75_120_20']

In [15]:
cross_validation_scores = get_cross_val_output(
    train_validation_data_one_hot, 
    X_columns=['LogGrLivArea', 'MSSubClass'],
    X_column_transform_map={'MSSubClass' : get_MSSubClass_indicator_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.22579781 0.23617559 0.25318008 0.23363783 0.24888801]
0.23953586670079288
0.01008404371911168


Now, let us how randomization changes the equation.

In [16]:
cross_validation_scores = get_cross_val_output(
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass'],
    X_column_transform_map={'MSSubClass' : get_MSSubClass_indicator_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.24114058 0.23456644 0.22777514 0.25112545 0.24581746]
0.24008501348231376
0.008220143850038789


Now, let us try the a transformation of  MSSubClass variable.

In [17]:
def get_mean_count_per_group(train_data, group_col):
    results_df = pd.DataFrame(train_data[['SalePrice', group_col]].groupby([group_col]).size())
    results_df['mean_SalePrice'] =  train_data[['SalePrice', group_col]].groupby(group_col).SalePrice.mean()
    results_df['mean_LogSalePrice'] =  train_data[['LogSalePrice', group_col]].groupby(group_col).LogSalePrice.mean()

  
    # Added later one.
    if 'LogSalePricePerSqFeet' in train_data.columns:
        results_df['mean_LogSalePricePerSqFeet'] =  \
            train_data[['LogSalePricePerSqFeet', group_col]].groupby(group_col).LogSalePricePerSqFeet.mean()    
        results_df.columns = ['Count', 'mean_SalePrice', 'mean_LogSalePrice', 'mean_LogSalePricePerSqFeet']
    else:
        results_df.columns = ['Count', 'mean_SalePrice', 'mean_LogSalePrice']
    results_df['percent_total_size'] = results_df['Count'] * 100.0/len(train_data)
    return results_df

In [18]:
def get_Group_LogSalePrice_transformation(group_col, train_data, validation_data):
    results_df = get_mean_count_per_group(train_data, group_col)    
    subclass_to_LogSalePrice = dict(zip(results_df.index, results_df.mean_LogSalePrice)) 
    
    train_data[group_col + '_Val'] = train_data[group_col].apply (lambda x : subclass_to_LogSalePrice.get(x))
    validation_data[group_col + '_Val'] = validation_data[group_col].apply (lambda x : subclass_to_LogSalePrice.get(x, 0))    
    return [group_col + '_Val']


In [19]:
def get_Group_LogSalePricePerSqFeet_transformation(group_col, train_data, validation_data):
    results_df = get_mean_count_per_group(train_data, group_col)    
    subclass_to_LogSalePricePerSqFeet = dict(zip(results_df.index, results_df.mean_LogSalePricePerSqFeet)) 
    
    group_val_col = group_col + '_Val'
    train_data[group_val_col] = \
        train_data[group_col].apply (lambda x : subclass_to_LogSalePricePerSqFeet.get(x))
    validation_data[group_val_col] = \
        validation_data[group_col].apply (lambda x : subclass_to_LogSalePricePerSqFeet.get(x, 0))    
    return [group_val_col]


Let us fix it up once for all by copying raw categorical columns to the dataframe having one hot encoded columns as well, so that we have just one data frame for reference.

In [20]:
raw_cat_cols = [x for x in train_validation_data.columns if x not in train_validation_data_one_hot.columns]

In [21]:
raw_cat_cols

['MSSubClass',
 'MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'MoSold',
 'YrSold',
 'SaleType',
 'SaleCondition']

In [22]:
for raw_cat_col in raw_cat_cols:
    train_validation_data_one_hot[raw_cat_col] = train_validation_data[raw_cat_col]

In [23]:
cross_validation_scores = get_cross_val_output(
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.27265148 0.2567731  0.23475537 0.25561167 0.23954792]
0.25186790645695256
0.013525196049074168


In [24]:
cross_validation_scores = get_cross_val_output(
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.21528763 0.22665061 0.21416977 0.23390888 0.21083384]
0.22017014572548593
0.00869205580238547


Add OverallQual and then OverallCond

In [25]:
cross_validation_scores = get_cross_val_output(
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.16796611 0.20909814 0.17741441 0.17144673 0.16397446]
0.1779799680160849
0.016172663512325637


In [26]:
cross_validation_scores = get_cross_val_output(
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.1641142  0.18310905 0.16295814 0.17056848 0.18793507]
0.1737369868416132
0.010082084596533133


Add Neighbourhood column.

In [27]:
cross_validation_scores = get_cross_val_output(
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Neighborhood' : get_Group_LogSalePricePerSqFeet_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.18157758 0.15239293 0.14661197 0.14135772 0.16710222]
0.15780848393104
0.014674201632321256


In [28]:
cross_validation_scores = get_cross_val_output(  
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.17194256 0.15724066 0.14460259 0.17031988 0.14274758]
0.15737065470215267
0.012303637393484635


Let us add the condition variable which we saw in the prior notebook.

In [29]:
def get_condition_to_logSalePrice(train_data):
    condition_to_logSalePrice = dict()
    conditions = train_data['Condition1'].unique()
    for condition in conditions:
        #print(condition)
        avg_logSalePrice = \
            train_data[
                (train_data['Condition1'] == condition) | 
                (train_data['Condition2'] == condition) ]['LogSalePrice'].mean()
        #print(avg_logSalePrice)
        condition_to_logSalePrice[condition] = avg_logSalePrice
    return condition_to_logSalePrice

In [30]:
def process_conditions(condition_1, condition_2, condition_to_logSalePrice):
    return (condition_to_logSalePrice.get(condition_1) + condition_to_logSalePrice.get(condition_2))

In [31]:
def get_Condition_LogSalePrice_transformation(group_col, train_data, validation_data):
    condition_to_LogSalePrice = get_condition_to_logSalePrice(train_data)
    
    group_val_col = group_col + '_Val'
    train_data[group_val_col] = \
        train_data['Condition1'].apply(lambda x : condition_to_LogSalePrice.get(x)) + \
        train_data['Condition2'].apply(lambda x : condition_to_LogSalePrice.get(x))
    validation_data[group_val_col] = \
        validation_data['Condition1'].apply(lambda x : condition_to_LogSalePrice.get(x)) + \
        validation_data['Condition2'].apply(lambda x : condition_to_LogSalePrice.get(x))

    return [group_val_col]


In [32]:
cross_validation_scores = get_cross_val_output(
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood', 'Condition'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Condition' : get_Condition_LogSalePrice_transformation,
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.14808679 0.16411915 0.178889   0.15938995 0.13723748]
0.15754447345515132
0.014170692425645209


Okay, that does not look to be much of a help. Let us see if just segregating norm conditions alone would help.

In [33]:
def get_norm_conditions_alone(group_col, train_data, validation_data):
    target_col = group_col + '_Norm'
    train_data[target_col] = train_data['Condition1_Norm'] + train_data['Condition2_Norm'] 
    validation_data[target_col] = validation_data['Condition1_Norm'] + validation_data['Condition2_Norm'] 
    return [target_col]

In [34]:
cross_validation_scores = get_cross_val_output(
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Condition' : get_norm_conditions_alone,
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.14894973 0.16312335 0.14585031 0.16224156 0.16888194]
0.15780937899335284
0.008854342088055784


In [35]:
(train_validation_data_one_hot['SalePrice'].isnull()).any() 

False

Let us pass on this variable for now and have a look BlgType

In [36]:
results_df = get_mean_count_per_group(train_validation_data, 'BldgType')

In [37]:
results_df.sort_values(['mean_LogSalePricePerSqFeet'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
BldgType,,,,,
TwnhsE,114,181959.342105,12.059457,4.907253,7.808219
1Fam,1220,185763.807377,12.047519,4.765520,83.561644
Twnhs,43,135911.627907,11.773914,4.655453,2.945205
Duplex,52,133541.076923,11.780928,4.464481,3.561644
2fmCon,31,128432.258065,11.725362,4.432202,2.123288


Almost 83% of the buildings are single family detached houses and I am not sure whether this would give us any benefit here.


In [38]:
cross_validation_scores = get_cross_val_output(
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood', 'BldgType'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'BldgType' : get_Group_LogSalePrice_transformation,
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.17311662 0.15990386 0.15371939 0.1694849  0.13592753]
0.15843046174346193
0.0131805223161619


Nothing impressive here. Let us check out house style here.

In [39]:
results_df = get_mean_count_per_group(train_validation_data, 'HouseStyle')
results_df.sort_values(['mean_LogSalePricePerSqFeet'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
HouseStyle,,,,,
SFoyer,37,135074.486486,11.786487,4.940404,2.534247
1Story,726,175985.477961,11.993690,4.858106,49.726027
SLvl,65,166703.384615,12.002160,4.807808,4.452055
1.5Unf,14,110150.000000,11.595037,4.802181,0.958904
2Story,445,210051.764045,12.182692,4.673151,30.479452
1.5Fin,154,143116.740260,11.813536,4.491929,10.547945
2.5Unf,11,157354.545455,11.908758,4.380611,0.753425
2.5Fin,8,220000.000000,12.194456,4.260308,0.547945


The distribution of values look more spread out, possibly indicating that this information is more useful. Let us check out.

In [40]:
cross_validation_scores = get_cross_val_output(
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood', 'HouseStyle'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'HouseStyle' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.13679468 0.17897863 0.14326391 0.40675698 0.17076828]
0.2073124956873294
0.10098650771831377


Again, this is rather disappointing here. Let us drill down on the age of the home here. This should give us something definitive.

Let us have a first look at the relevant variables here.

In [41]:
train_validation_data['YrSold'].unique()

array([2008, 2007, 2006, 2009, 2010])

In [42]:
train_validation_data['YearBuilt'].unique()

array([2003, 1976, 2001, 1915, 2000, 1993, 2004, 1973, 1931, 1939, 1965,
       2005, 1962, 2006, 1960, 1929, 1970, 1967, 1958, 1930, 2002, 1968,
       2007, 1951, 1957, 1927, 1920, 1966, 1959, 1994, 1954, 1953, 1955,
       1983, 1975, 1997, 1934, 1963, 1981, 1964, 1999, 1972, 1921, 1945,
       1982, 1998, 1956, 1948, 1910, 1995, 1991, 2009, 1950, 1961, 1977,
       1985, 1979, 1885, 1919, 1990, 1969, 1935, 1988, 1971, 1952, 1936,
       1923, 1924, 1984, 1926, 1940, 1941, 1987, 1986, 2008, 1908, 1892,
       1916, 1932, 1918, 1912, 1947, 1925, 1900, 1980, 1989, 1992, 1949,
       1880, 1928, 1978, 1922, 1996, 2010, 1946, 1913, 1937, 1942, 1938,
       1974, 1893, 1914, 1906, 1890, 1898, 1904, 1882, 1875, 1911, 1917,
       1872, 1905])

In [43]:
train_validation_data['YearRemodAdd'].unique()

array([2003, 1976, 2002, 1970, 2000, 1995, 2005, 1973, 1950, 1965, 2006,
       1962, 2007, 1960, 2001, 1967, 2004, 2008, 1997, 1959, 1990, 1955,
       1983, 1980, 1966, 1963, 1987, 1964, 1972, 1996, 1998, 1989, 1953,
       1956, 1968, 1981, 1992, 2009, 1982, 1961, 1993, 1999, 1985, 1979,
       1977, 1969, 1958, 1991, 1971, 1952, 1975, 2010, 1984, 1986, 1994,
       1988, 1954, 1957, 1951, 1978, 1974])

In [44]:
len(train_validation_data)

1460

In [45]:
train_validation_data[
    train_validation_data['YearBuilt'] == train_validation_data['YearRemodAdd']][['YearBuilt', 'YearRemodAdd']]

,YearBuilt,YearRemodAdd
0,2003,2003
1,1976,1976
4,2000,2000
7,1973,1973
10,1965,1965
12,1962,1962
14,1960,1960
16,1970,1970
17,1967,1967
18,2004,2004


In [46]:
train_validation_data[
    train_validation_data['YearBuilt'] != train_validation_data['YearRemodAdd']][['YearBuilt', 'YearRemodAdd']]

,YearBuilt,YearRemodAdd
2,2001,2002
3,1915,1970
5,1993,1995
6,2004,2005
8,1931,1950
9,1939,1950
11,2005,2006
13,2006,2007
15,1929,2001
19,1958,1965


Since, the remodelling could improve the home a lot, let us take the remodeling data as the 'birhtdate' of the home and calculate a synthetic age for the home and see how it helps us.

In [47]:
train_validation_data_one_hot['YrSold'] = train_validation_data['YrSold']

In [48]:
(train_validation_data_one_hot['YrSold'] - train_validation_data_one_hot['YearRemodAdd']).unique()

array([ 5, 31,  6, 36,  8, 14,  2, 58, 43,  0, 46, 48, 40, 39,  4, 44, 57,
        9, 10,  1, 51, 19,  3, 53, 41, 24, 28, 47, 56, 52, 11, 60, 21, 34,
        7, 42, 13, 27, 12, 45, 16, 25, 30, 33, 54, 20, 37, 50, 15, 38, 17,
       32, 35, 22, 18, 59, 29, 49, 26, -1, 55, 23])

In [49]:
train_validation_data_one_hot['Age'] = \
    (train_validation_data_one_hot['YrSold'] - train_validation_data_one_hot['YearRemodAdd'])

In [50]:
train_validation_data_one_hot.loc[(train_validation_data_one_hot['Age'] == -1), 'Age'] = 0

In [51]:
train_validation_data_one_hot['Age'].unique()

array([ 5, 31,  6, 36,  8, 14,  2, 58, 43,  0, 46, 48, 40, 39,  4, 44, 57,
        9, 10,  1, 51, 19,  3, 53, 41, 24, 28, 47, 56, 52, 11, 60, 21, 34,
        7, 42, 13, 27, 12, 45, 16, 25, 30, 33, 54, 20, 37, 50, 15, 38, 17,
       32, 35, 22, 18, 59, 29, 49, 26, 55, 23])

In [52]:
results_df = get_mean_count_per_group(train_validation_data_one_hot, 'Age')

In [53]:
results_df

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
Age,,,,,
0,125,268117.328000,12.423553,4.985692,8.561644
1,87,221081.931034,12.232200,4.922563,5.958904
2,52,213300.769231,12.209725,4.905932,3.561644
3,53,197014.150943,12.145477,4.842380,3.630137
4,63,203769.444444,12.175371,4.889054,4.315068
5,46,189917.391304,12.080414,4.838733,3.150685
6,40,200525.375000,12.161284,4.809135,2.739726
7,38,193372.921053,12.115680,4.780427,2.602740
8,42,209098.809524,12.209377,4.795576,2.876712


In [54]:
results_df.sort_values(['mean_LogSalePricePerSqFeet'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
Age,,,,,
0,125,268117.328000,12.423553,4.985692,8.561644
1,87,221081.931034,12.232200,4.922563,5.958904
2,52,213300.769231,12.209725,4.905932,3.561644
4,63,203769.444444,12.175371,4.889054,4.315068
24,6,130166.666667,11.754996,4.847407,0.410959
19,11,205268.181818,12.191814,4.847368,0.753425
3,53,197014.150943,12.145477,4.842380,3.630137
25,7,177285.714286,12.067559,4.839341,0.479452
5,46,189917.391304,12.080414,4.838733,3.150685


There definitely looks to be a linear relationship here (with the exception of few outliers). Let us try it in our model.

In [55]:
cross_validation_scores = get_cross_val_output(
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood', 'Age'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'HouseStyle' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.14410925 0.16495578 0.15695884 0.15385146 0.1639492 ]
0.1567649056996032
0.007581500640605678


We tend to feel that we are not extracting as much predictive juice from 'Age' variable as we should. Let us try bucketizing this variable and see if we can squeeze out more.

In [56]:
def get_age_category(age):
    if age < 5:
        category = '0-5'
    elif age < 10:
        category = '5-10'
    elif age < 15:
        category = '10-15'
    elif age < 20:
        category = '15-20'
    elif age < 30:
        category = '20-30'
    elif age < 35:
        category = '30-35'
    elif age < 40:
        category = '35-40'
    elif age < 45:
        category = '40-45'
    elif age < 50:
        category = '45-50'
    elif age < 55:
        category = '50-55'
    elif age < 60:
        category = '55-60'

    else:
        category = '>60'
    return category

In [57]:
train_validation_data_one_hot['Age_Category'] = train_validation_data_one_hot['Age'].apply(lambda x : get_age_category(x))

In [58]:
results_df = get_mean_count_per_group(train_validation_data_one_hot, 'Age_Category')

In [59]:
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
Age_Category,,,,,
0-5,380,229262.260526,12.270552,4.924314,26.027397
10-15,150,213389.246667,12.187037,4.759845,10.273973
5-10,204,197034.411765,12.139308,4.798591,13.972603
15-20,75,192693.733333,12.126393,4.753002,5.136986
20-30,85,168464.823529,11.999060,4.728818,5.821918
30-35,92,156831.423913,11.934730,4.703756,6.301370
40-45,68,149755.926471,11.877892,4.705577,4.657534
45-50,65,138125.107692,11.820983,4.722852,4.452055
35-40,88,134197.602273,11.769491,4.690101,6.027397


Let us try out this variable and see how it goes.

In [60]:
cross_validation_scores = get_cross_val_output(
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood', 'Age_Category'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Age_Category' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.16066777 0.14260864 0.13321329 0.18891531 0.15178197]
0.15543739500235107
0.019078667341301436


Now that we have explored these many features pertaining to fundamental price value, let us take break and explore other features pertaining to the transaction (Say sale type etc). 

This will help us get some orthogonal factors than what we have explored so far.

In [61]:
results_df = get_mean_count_per_group(train_validation_data, 'SaleType')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
SaleType,,,,,
Con,2,269600.000000,12.483915,5.086516,0.136986
New,122,274945.418033,12.459376,5.003455,8.356164
CWD,4,210600.000000,12.198350,4.719931,0.273973
ConLI,5,200390.000000,12.044885,4.690562,0.342466
WD,1267,173401.836622,11.991068,4.738893,86.780822
COD,43,143973.255814,11.827445,4.594120,2.945205
ConLD,9,138780.888889,11.773009,4.579898,0.616438
ConLw,5,143700.000000,11.769714,4.623670,0.342466
Oth,3,119850.000000,11.675303,4.702550,0.205479


There looks to be some value in a home that is constructed and sold immeidately. But shouldn't that corresond to age=0 ?

In [62]:
train_validation_data_one_hot['SaleType'] = train_validation_data['SaleType']

In [63]:
train_validation_data_one_hot[
    (train_validation_data_one_hot['SaleType'] == 'New') & (train_validation_data_one_hot['Age'] !=0 )]

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,...,PavedDrive,PoolQC,Fence,MiscFeature,MoSold,YrSold,SaleType,SaleCondition,Age,Age_Category
48,736,716,0,2,0.0,0.0,0.0,0.0,736.0,102,...,N,Missing,Missing,Missing,6,2009,New,Partial,1,0-5
60,1158,0,0,3,941.0,0.0,1.0,0.0,217.0,0,...,Y,Missing,Missing,Missing,5,2006,New,Partial,2,0-5
162,1541,0,0,3,1201.0,0.0,0.0,0.0,340.0,0,...,Y,Missing,Missing,Missing,5,2010,New,Partial,5,5-10
401,1310,0,0,3,24.0,0.0,0.0,0.0,1286.0,0,...,Y,Missing,Missing,Missing,7,2006,New,Partial,1,0-5
408,1071,1101,0,3,0.0,0.0,0.0,0.0,1063.0,0,...,Y,Missing,Missing,Missing,8,2007,New,Partial,1,0-5
412,1478,0,0,2,578.0,0.0,1.0,0.0,892.0,0,...,Y,Missing,Missing,Missing,6,2010,New,Partial,1,0-5
678,2046,0,0,3,0.0,0.0,0.0,0.0,2046.0,0,...,Y,Missing,Missing,Missing,7,2009,New,Partial,1,0-5
774,1973,0,0,3,0.0,0.0,0.0,0.0,1935.0,0,...,Y,Missing,Missing,Missing,7,2007,New,Partial,1,0-5
1344,728,728,0,3,0.0,0.0,0.0,0.0,728.0,0,...,Y,Missing,Missing,Missing,7,2007,New,Partial,1,0-5


In [64]:
train_validation_data_one_hot[
    (train_validation_data_one_hot['SaleType'] != 'New') & (train_validation_data_one_hot['Age'] ==0 )]

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,...,PavedDrive,PoolQC,Fence,MiscFeature,MoSold,YrSold,SaleType,SaleCondition,Age,Age_Category
145,970,739,0,3,0.0,0.0,0.0,0.0,970.0,0,...,Y,Missing,Missing,Missing,4,2006,WD,Normal,0,0-5
251,1625,0,0,2,1573.0,0.0,1.0,1.0,0.0,0,...,Y,Missing,Missing,Missing,12,2007,WD,Family,0,0-5
568,1496,636,0,1,1441.0,0.0,1.0,0.0,55.0,0,...,Y,Missing,Missing,Missing,9,2009,WD,Normal,0,0-5
789,976,1111,0,5,568.0,0.0,0.0,0.0,264.0,0,...,Y,Missing,Missing,Missing,7,2007,WD,Normal,0,0-5
855,1040,0,0,3,659.0,0.0,1.0,0.0,381.0,0,...,Y,Missing,Missing,Missing,4,2010,WD,Normal,0,0-5
1158,1580,0,0,3,0.0,0.0,0.0,0.0,1580.0,0,...,Y,Missing,Missing,Missing,6,2008,ConLD,Partial,0,0-5
1227,912,0,0,2,595.0,0.0,1.0,0.0,317.0,0,...,Y,Missing,Missing,Missing,12,2008,WD,Normal,0,0-5
1327,816,0,0,3,641.0,0.0,0.0,1.0,175.0,0,...,Y,Missing,MnPrv,Missing,10,2008,WD,Normal,0,0-5
1383,1416,0,0,3,0.0,0.0,0.0,0.0,816.0,112,...,N,Missing,Missing,Missing,8,2007,WD,Normal,0,0-5
1403,1464,0,0,3,929.0,0.0,1.0,0.0,556.0,0,...,Y,Missing,Missing,Missing,8,2007,WD,Normal,0,0-5


Looks like there is a huge overlap here. Anyways, let us try to see if this information helps us.

In [65]:
cross_validation_scores = get_cross_val_output(
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood', 'SaleType_New'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Age_Category' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.17830334 0.1432092  0.1714937  0.13934052 0.15181772]
0.1568328961624972
0.015444478477900401


As expected, we do not get much of benefit here , as this information was alread present as part of age variable, which we did not find useful.

In [66]:
results_df = get_mean_count_per_group(train_validation_data, 'SaleCondition')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
SaleCondition,,,,,
Partial,125,272291.752000,12.447481,4.999833,8.561644
Normal,1198,175202.219533,12.005040,4.749434,82.054795
Alloca,12,167377.416667,11.914465,4.538400,0.821918
Family,20,149600.000000,11.869662,4.599566,1.369863
Abnorml,101,146526.623762,11.788783,4.591861,6.917808
AdjLand,4,104125.000000,11.529157,4.516512,0.273973


Does Partial sale condition help ?

In [67]:
train_validation_data_one_hot['SaleCondition_Normal_or_Partial'] = \
    train_validation_data_one_hot['SaleCondition_Normal'] + train_validation_data_one_hot['SaleCondition_Partial']
test_data_one_hot['SaleCondition_Normal_or_Partial'] = \
    test_data_one_hot['SaleCondition_Normal'] + test_data_one_hot['SaleCondition_Partial']    

In [68]:
cross_validation_scores = get_cross_val_output(
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood', 'SaleCondition_Normal_or_Partial'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Age_Category' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.16004098 0.15747915 0.14754264 0.16215742 0.15608715]
0.15666146815766452
0.005015535123202938


Nothing much significant here.

Let us have a look at fence variable here.

In [69]:
results_df = get_mean_count_per_group(train_validation_data, 'Fence')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
Fence,,,,,
Missing,1179,187596.837998,12.058605,4.769635,80.753425
GdPrv,59,178927.457627,12.057119,4.707527,4.041096
MnPrv,157,148751.089172,11.850604,4.700378,10.753425
MnWw,11,134286.363636,11.796808,4.764014,0.753425
GdWo,54,140379.314815,11.784241,4.659067,3.698630


In [70]:
cross_validation_scores = get_cross_val_output(
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood', 'Fence_Missing'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Age_Category' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.39709635 0.156149   0.16015401 0.17267417 0.14879231]
0.20697316913111527
0.09537650382896172


In [71]:
results_df = get_mean_count_per_group(train_validation_data, 'PoolQC')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
PoolQC,,,,,
Ex,2,490000.000000,12.944243,4.719218,0.136986
Fa,2,215500.000000,12.267739,4.712846,0.136986
Gd,3,201990.000000,12.185594,4.400433,0.205479
Missing,1453,180404.663455,12.022122,4.756388,99.520548


In [72]:
train_validation_data_one_hot['BedroomAbvGr'].describe()

count    1460.000000
mean        2.866438
std         0.815778
min         0.000000
25%         2.000000
50%         3.000000
75%         3.000000
max         8.000000
Name: BedroomAbvGr, dtype: float64

In [73]:
cross_validation_scores = get_cross_val_output(
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood', 'BedroomAbvGr'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Age_Category' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.14822693 0.17792703 0.16330308 0.14836772 0.15179961]
0.15792487289922902
0.011418576048520456


The price could be influenced by the general market conditions during that time frame and that could be quantified by the volume sold in a particular year. Let us check out the stats per year.

In [74]:
results_df = get_mean_count_per_group(train_validation_data, 'YrSold')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
YrSold,,,,,
2007,329,186063.151976,12.050674,4.766450,22.534247
2006,314,182549.458599,12.034991,4.754475,21.506849
2008,304,177360.838816,12.014212,4.763298,20.821918
2009,338,179432.103550,12.008744,4.739883,23.150685
2010,175,177393.674286,12.001079,4.753751,11.986301


Let us see if this helps

In [75]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood', 'YrSold'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.1523694  0.15157312 0.15614098 0.16361999 0.16839574]
0.15841984545214477
0.006561077286557759


Not much of benefit. Let us take a look at MiscVal

In [76]:
results_df = get_mean_count_per_group(train_validation_data, 'MiscFeature')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
MiscFeature,,,,,
TenC,1,250000.000000,12.429220,4.497217,0.068493
Gar2,2,170750.000000,12.041566,4.672965,0.136986
Missing,1406,182046.410384,12.030119,4.758545,96.301370
Shed,49,151187.612245,11.868394,4.697333,3.356164
Othr,2,94000.000000,11.356609,4.285276,0.136986


Let us try GarageArea

In [77]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood', 'GarageArea'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.13369542 0.1530005  0.14386942 0.17319973 0.15998325]
0.15274966307865584
0.013509376059756285


There looks to be some benefit here. Let us check out other garage related variables here.

In [78]:
results_df = get_mean_count_per_group(train_validation_data, 'GarageQual')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
GarageQual,,,,,
Gd,14,215860.714286,12.220073,4.779462,0.958904
Ex,3,241000.000000,12.175455,4.971479,0.205479
TA,1311,187489.836003,12.068546,4.781381,89.794521
Fa,48,123573.354167,11.674854,4.496699,3.287671
Missing,81,103317.283951,11.491885,4.491847,5.547945
Po,3,100166.666667,11.472505,4.399287,0.205479


Does not look promising as 89% of the values are typical

In [79]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood', 'GarageArea', 'GarageQual'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'GarageQual' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.13898404 0.16796242 0.16605347 0.13864652 0.14900488]
0.15213026526069284
0.012719398217235795


In [80]:
results_df = get_mean_count_per_group(train_validation_data, 'GarageCond')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
GarageCond,,,,,
TA,1326,187885.735294,12.070489,4.781517,90.821918
Gd,9,179930.000000,12.050621,4.753300,0.616438
Ex,2,124000.000000,11.727646,5.017276,0.136986
Fa,35,114654.028571,11.596890,4.447904,2.397260
Po,7,108500.000000,11.572888,4.353483,0.479452
Missing,81,103317.283951,11.491885,4.491847,5.547945


The same conclusion as before.

In [81]:
train_validation_data_one_hot['GarageCars']

0       2.0
1       2.0
2       2.0
3       3.0
4       3.0
5       2.0
6       2.0
7       2.0
8       2.0
9       1.0
10      1.0
11      3.0
12      1.0
13      3.0
14      1.0
15      2.0
16      2.0
17      2.0
18      2.0
19      1.0
20      3.0
21      1.0
22      2.0
23      2.0
24      1.0
25      3.0
26      2.0
27      3.0
28      1.0
29      1.0
       ... 
1430    2.0
1431    2.0
1432    1.0
1433    2.0
1434    2.0
1435    2.0
1436    2.0
1437    3.0
1438    2.0
1439    2.0
1440    2.0
1441    2.0
1442    3.0
1443    1.0
1444    2.0
1445    1.0
1446    1.0
1447    2.0
1448    1.0
1449    0.0
1450    0.0
1451    3.0
1452    2.0
1453    0.0
1454    2.0
1455    2.0
1456    2.0
1457    1.0
1458    1.0
1459    1.0
Name: GarageCars, Length: 1460, dtype: float64

In [82]:
results_df = get_mean_count_per_group(train_validation_data, 'GarageCars')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
GarageCars,,,,,
3,181,309636.121547,12.583594,4.974925,12.397260
4,5,192655.800000,12.137204,4.712470,0.342466
2,824,183851.663835,12.083619,4.774973,56.438356
1,369,128116.688347,11.731876,4.663024,25.273973
0,81,103317.283951,11.491885,4.491847,5.547945


This looks interesting. Let us try it out.

In [83]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood', 'GarageArea', 'GarageCars'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'GarageQual' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.14077368 0.16056561 0.15808522 0.16071573 0.14395019]
0.15281808413781253
0.008646929555323039


It  looks like GarageArea and GarageCars are essentially giving us the same information (it rather makes sense as well).

In [84]:
train_validation_data_one_hot['GarageCars_2_or_3'] = \
    (train_validation_data_one_hot['GarageCars'] == 2) | (train_validation_data_one_hot['GarageCars'] == 3)

In [85]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood', 'GarageArea'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'GarageQual' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.15248128 0.15803901 0.15868917 0.13733877 0.1552429 ]
0.15235822635011992
0.00782652711558512


In [86]:
results_df = get_mean_count_per_group(train_validation_data, 'GarageFinish')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
GarageFinish,,,,,
Fin,352,240052.690341,12.320815,4.882571,24.109589
RFn,422,202068.869668,12.172678,4.840549,28.904110
Unf,605,142156.423140,11.818982,4.657653,41.438356
Missing,81,103317.283951,11.491885,4.491847,5.547945


In [87]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood', 'GarageArea', 'GarageFinish'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'GarageQual' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'GarageFinish' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.14021921 0.1558085  0.14904222 0.14558294 0.17072741]
0.1522760539178325
0.01052191744617027


In [88]:
results_df = get_mean_count_per_group(train_validation_data, 'GarageType')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
GarageType,,,,,
BuiltIn,88,254751.738636,12.374923,4.760318,6.027397
Attchd,870,202892.656322,12.160783,4.839228,59.589041
Basment,19,160570.684211,11.916447,4.632503,1.301370
2Types,6,151283.333333,11.899164,4.502868,0.410959
Detchd,387,134091.162791,11.765651,4.639229,26.506849
CarPort,9,109962.111111,11.588004,4.422668,0.616438
Missing,81,103317.283951,11.491885,4.491847,5.547945


In [89]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood', 'GarageArea', 'GarageType'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'GarageType' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'GarageFinish' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.14720089 0.15687386 0.15596616 0.15155325 0.14941182]
0.15220119660623788
0.0037205444584749265


Let us try the case of Basement Square Feet.

In [90]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 'Neighborhood', 'GarageArea', 'TotalBsmtSF'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'GarageType' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'GarageFinish' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.18764927 0.14601286 0.13731661 0.14202893 0.37722508]
0.19804654881919537
0.09137467147027482


In [91]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 
               'Neighborhood', 'GarageArea', 'TotalBsmtSF'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'GarageType' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'GarageFinish' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.15422619 0.17456546 0.13578342 0.39232552 0.13642535]
0.1986651889117402
0.09786506883350846


In [92]:
results_df = get_mean_count_per_group(train_validation_data, 'BsmtQual')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
BsmtQual,,,,,
Ex,121,327041.041322,12.640397,5.054466,8.287671
Gd,618,202688.478964,12.179888,4.837848,42.328767
TA,649,140759.818182,11.810863,4.654034,44.452055
Fa,35,115692.028571,11.617609,4.495547,2.397260
Missing,37,105652.891892,11.529691,4.429834,2.534247


In [93]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 
               'Neighborhood', 'GarageArea', 'BsmtQual'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'BsmtQual' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'GarageQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageFinish' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.14196322 0.14450021 0.15709107 0.15280536 0.14774259]
0.1488204915290142
0.005497826301219295


In [94]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 
               'Neighborhood', 'GarageArea', 'BsmtQual'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'BsmtQual' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'KitchenQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageFinish' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.15073992 0.13570944 0.1567803  0.14258941 0.15381061]
0.14792593756891131
0.007728764560582634


In [95]:
train_validation_data_one_hot['BedroomAbvGr']

0       3
1       3
2       3
3       3
4       4
5       1
6       3
7       3
8       2
9       2
10      3
11      4
12      2
13      3
14      2
15      2
16      2
17      2
18      3
19      3
20      4
21      3
22      3
23      3
24      3
25      3
26      3
27      3
28      2
29      1
       ..
1430    4
1431    2
1432    4
1433    3
1434    3
1435    3
1436    3
1437    2
1438    2
1439    3
1440    3
1441    1
1442    3
1443    2
1444    3
1445    3
1446    3
1447    3
1448    2
1449    1
1450    4
1451    3
1452    2
1453    3
1454    2
1455    3
1456    3
1457    4
1458    2
1459    3
Name: BedroomAbvGr, Length: 1460, dtype: int64

In [96]:
results_df = get_mean_count_per_group(train_validation_data, 'BedroomAbvGr')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
BedroomAbvGr,,,,,
0,6,221493.166667,12.209000,5.010310,0.410959
8,1,200000.000000,12.206078,4.075724,0.068493
4,213,220421.253521,12.204733,4.593217,14.589041
3,804,181056.870647,12.049888,4.772688,55.068493
5,21,180819.047619,12.035595,4.296783,1.438356
1,50,173162.420000,11.893030,4.963170,3.424658
2,358,158197.659218,11.875708,4.817960,24.520548
6,7,143779.000000,11.863333,4.305955,0.479452


In [97]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 
               'Neighborhood', 'GarageArea', 'BsmtQual'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'BsmtQual' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'KitchenQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageFinish' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.1645964  0.1359706  0.1542574  0.14819461 0.1387749 ]
0.1483587845552799
0.010428792734122753


Let us take a look at foundation variable.

In [98]:
results_df = get_mean_count_per_group(train_validation_data, 'Foundation')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
Foundation,,,,,
PConc,647,225230.442040,12.261670,4.871669,44.315068
Wood,3,185666.666667,12.102485,4.596967,0.205479
Stone,6,165959.166667,11.933163,4.456350,0.410959
CBlock,634,149805.714511,11.870087,4.710342,43.424658
BrkTil,146,132291.075342,11.722536,4.509594,10.000000
Slab,24,107365.625000,11.532970,4.410014,1.643836


In [99]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot.sample(frac=1), 
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 
               'Neighborhood', 'GarageArea', 'BsmtQual', 'Foundation'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Foundation' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'BsmtQual' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'KitchenQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageFinish' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.14252025 0.15684237 0.16239294 0.15687278 0.14065817]
0.15185730116875204
0.008644148816955077


Not much of benefit here. Let us check out variables reflecting exterior condition.

In [100]:
results_df = get_mean_count_per_group(train_validation_data, 'ExterQual')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
ExterQual,,,,,
Ex,52,367360.961538,12.764047,5.089239,3.561644
Gd,488,231633.510246,12.311287,4.884112,33.424658
TA,906,144341.313466,11.837993,4.673155,62.054795
Fa,14,87985.214286,11.304554,4.366474,0.958904


In [101]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot.sample(frac=1),  
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 
               'Neighborhood', 'GarageArea', 'BsmtQual', 'ExterQual'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'ExterQual' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'BsmtQual' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'KitchenQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageFinish' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())


[0.13963421 0.15001213 0.16653057 0.14756928 0.13722488]
0.1481942136325714
0.010329736524151806


In [102]:
results_df = get_mean_count_per_group(train_validation_data, 'ExterCond')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
ExterCond,,,,,
Ex,3,201333.333333,12.119733,4.883889,0.205479
TA,1282,184034.896256,12.043084,4.767244,87.808219
Gd,146,168897.568493,11.969457,4.731892,10.000000
Fa,28,102595.142857,11.455174,4.354016,1.917808
Po,1,76500.000000,11.245059,4.069569,0.068493


This does not show much of benefit, but let us test it out anyways.

I am omitting this for now, as this causes technical issues with ExteriorCond 'Po' being present in one data set and not in the other.

Let us check out MasVnrArea


In [103]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot.sample(frac=1),  
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 
               'Neighborhood', 'GarageArea', 'BsmtQual', 'LogMasVnrArea_times_not_missing'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'ExterQual' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'BsmtQual' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'KitchenQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageFinish' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())
 

[0.14599391 0.16834432 0.14192259 0.13677668 0.14748074]
0.14810364798580247
0.010779883757795356


In [104]:
results_df = get_mean_count_per_group(train_validation_data, 'MasVnrType')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
MasVnrType,,,,,
Stone,128,265583.625000,12.425142,4.980576,8.767123
Missing,8,236484.250000,12.325059,4.879385,0.547945
BrkFace,445,204691.871910,12.163635,4.804588,30.479452
None,864,156221.891204,11.892927,4.698656,59.178082
BrkCmn,15,146318.066667,11.853246,4.591163,1.027397


In [105]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot.sample(frac=1),  
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 
               'Neighborhood', 'GarageArea', 'BsmtQual', 'MasVnrType'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'MasVnrType' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'BsmtQual' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'KitchenQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageFinish' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())
 

[0.1306567  0.15915243 0.14402943 0.14648081 0.16265704]
0.14859528123228555
0.011456852549827317


In [106]:
results_df = get_mean_count_per_group(train_validation_data, 'Exterior2nd')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
Exterior2nd,,,,,
Other,1,319000.000000,12.672950,5.162519,0.068493
ImStucc,10,252070.000000,12.240061,4.871142,0.684932
VinylSd,504,214432.460317,12.209636,4.865528,34.520548
CmentBd,60,230093.833333,12.201954,4.853625,4.109589
BrkFace,25,195818.000000,12.066074,4.776934,1.712329
Plywood,142,168112.387324,11.994750,4.728172,9.726027
HdBoard,207,167661.565217,11.974939,4.753625,14.178082
Stone,5,158224.800000,11.898449,4.541362,0.342466
Wd Shng,38,161328.947368,11.892250,4.670004,2.602740


In [107]:
# We removed some entries that extremely few occurences so that they do not cause problems later in cross validation.
train_validation_data_one_hot_new = train_validation_data_one_hot.copy()
train_validation_data_one_hot_new = \
    train_validation_data_one_hot_new[(train_validation_data_one_hot_new['Exterior2nd'] != 'Other') &
                                      (train_validation_data_one_hot_new['Exterior2nd'] != 'CBlock') &
                                      (train_validation_data_one_hot_new['Exterior2nd'] != 'AsphShn')]

In [108]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot_new.sample(frac=1),  
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 
               'Neighborhood', 'GarageArea', 'BsmtQual', 'Exterior2nd'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Exterior2nd' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'BsmtQual' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'KitchenQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageFinish' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())
 

[0.13768015 0.15208998 0.14853364 0.15843695 0.14807057]
0.14896225836764146
0.006750070684288924


In [109]:
results_df = get_mean_count_per_group(train_validation_data, 'Exterior1st')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
Exterior1st,,,,,
ImStucc,1,262000.000000,12.476104,5.082841,0.068493
Stone,2,258500.000000,12.456540,4.719386,0.136986
VinylSd,515,213732.900971,12.205878,4.862191,35.273973
CemntBd,61,231690.655738,12.204928,4.849912,4.178082
BrkFace,50,194573.000000,12.088146,4.749935,3.424658
Plywood,108,175942.379630,12.041224,4.731604,7.397260
HdBoard,222,163077.450450,11.951905,4.759527,15.205479
Stucco,25,162990.000000,11.891999,4.527735,1.712329
MetalSd,220,149422.177273,11.859775,4.692604,15.068493


In [110]:
# We removed some entries that extremely few occurences so that they do not cause problems later in cross validation.
train_validation_data_one_hot_new = train_validation_data_one_hot.copy()
train_validation_data_one_hot_new = \
    train_validation_data_one_hot_new[(train_validation_data_one_hot_new['Exterior1st'] != 'ImStucc') &
                                      (train_validation_data_one_hot_new['Exterior1st'] != 'Stone') &
                                      (train_validation_data_one_hot_new['Exterior1st'] != 'CBlock') &
                                      (train_validation_data_one_hot_new['Exterior1st'] != 'BrkComm') &                                      
                                      (train_validation_data_one_hot_new['Exterior1st'] != 'AsphShn')]

In [111]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot_new.sample(frac=1),  
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 
               'Neighborhood', 'GarageArea', 'BsmtQual', 'Exterior1st'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Exterior1st' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'BsmtQual' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'KitchenQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageFinish' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())
 

[0.12860339 0.17357685 0.14504229 0.14775481 0.14081155]
0.14715778027009305
0.014746790139193411


Let us check out roof material column.

In [112]:
results_df = get_mean_count_per_group(train_validation_data, 'RoofMatl')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
RoofMatl,,,,,
WdShngl,6,390250.000000,12.763196,5.020048,0.410959
Membran,1,241500.000000,12.394629,5.176452,0.068493
WdShake,5,241400.000000,12.384854,4.718838,0.342466
Metal,1,180000.000000,12.100718,5.204023,0.068493
Tar&Grv,11,185406.363636,12.068084,4.672913,0.753425
CompShg,1434,179803.679219,12.019223,4.755919,98.219178
ClyTile,1,160000.000000,11.982935,3.344764,0.068493
Roll,1,137000.000000,11.827744,4.267142,0.068493


Not really much of point here, because we have more than 98% of the data having standard material

In [113]:
results_df = get_mean_count_per_group(train_validation_data, 'RoofStyle')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
RoofStyle,,,,,
Shed,2,225000.000000,12.311613,4.711082,0.136986
Hip,286,218876.933566,12.184435,4.831512,19.589041
Flat,13,194690.000000,12.122916,4.789976,0.890411
Mansard,7,180568.428571,12.057431,4.517038,0.479452
Gable,1141,171483.956179,11.984206,4.740600,78.150685
Gambrel,11,148909.090909,11.797505,4.449885,0.753425


In [114]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot.sample(frac=1),  
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 
               'Neighborhood', 'GarageArea', 'BsmtQual', 'RoofStyle'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'RoofStyle' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'BsmtQual' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'KitchenQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageFinish' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())
 

[0.1605602  0.15643863 0.13645742 0.13605874 0.15165732]
0.14823446354928677
0.01017737556095932


Not much of benefit here as well.

In [115]:
results_df = get_mean_count_per_group(train_validation_data, 'Functional')
results_df.sort_values(['mean_LogSalePrice'], ascending=False)

,Count,mean_SalePrice,mean_LogSalePrice,mean_LogSalePricePerSqFeet,percent_total_size
Functional,,,,,
Typ,1360,183429.147059,12.038518,4.776616,93.150685
Min1,31,146385.483871,11.862174,4.556759,2.123288
Maj1,14,153948.142857,11.859626,4.417754,0.958904
Mod,15,168393.333333,11.858680,4.435840,1.027397
Min2,34,144240.647059,11.845505,4.465472,2.328767
Sev,1,129000.000000,11.767575,3.992300,0.068493
Maj2,5,85800.000000,11.316568,4.286954,0.342466


In [116]:
# We removed some entries that extremely few occurences so that they do not cause problems later in cross validation.
train_validation_data_one_hot_new = train_validation_data_one_hot.copy()
train_validation_data_one_hot_new = \
    train_validation_data_one_hot_new[(train_validation_data_one_hot_new['Functional'] != 'Sev')]

In [117]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot_new.sample(frac=1),  
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 
               'Neighborhood', 'GarageArea', 'BsmtQual'],
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Functional' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'BsmtQual' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'KitchenQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageFinish' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())
 

[0.16395534 0.15260191 0.14414566 0.14003495 0.13543165]
0.14723390213935075
0.010090367288454007


Would things have changed if we have used the one hot encoded versions of MSSubClas/Neighborhood variables ?

In [118]:
MSSUBCLASS_ONE_HOT_COLS = [col for col in train_validation_data_one_hot.columns if 'MSSubClass_' in col]
NEIGHBORHOOD_ONE_HOT_COLS = [col for col in train_validation_data_one_hot.columns if 'Neighborhood_' in col]
BSMT_QUAL_ONE_HOT_COLS = [col for col in train_validation_data_one_hot.columns if 'BsmtQual_' in col]

In [119]:
NEIGHBORHOOD_ONE_HOT_COLS

['Neighborhood_Blmngtn',
 'Neighborhood_Blueste',
 'Neighborhood_BrDale',
 'Neighborhood_BrkSide',
 'Neighborhood_ClearCr',
 'Neighborhood_CollgCr',
 'Neighborhood_Crawfor',
 'Neighborhood_Edwards',
 'Neighborhood_Gilbert',
 'Neighborhood_IDOTRR',
 'Neighborhood_MeadowV',
 'Neighborhood_Mitchel',
 'Neighborhood_NAmes',
 'Neighborhood_NPkVill',
 'Neighborhood_NWAmes',
 'Neighborhood_NoRidge',
 'Neighborhood_NridgHt',
 'Neighborhood_OldTown',
 'Neighborhood_SWISU',
 'Neighborhood_Sawyer',
 'Neighborhood_SawyerW',
 'Neighborhood_Somerst',
 'Neighborhood_StoneBr',
 'Neighborhood_Timber',
 'Neighborhood_Veenker']

In [120]:
MSSUBCLASS_ONE_HOT_COLS

['MSSubClass_120',
 'MSSubClass_150',
 'MSSubClass_160',
 'MSSubClass_180',
 'MSSubClass_190',
 'MSSubClass_20',
 'MSSubClass_30',
 'MSSubClass_40',
 'MSSubClass_45',
 'MSSubClass_50',
 'MSSubClass_60',
 'MSSubClass_70',
 'MSSubClass_75',
 'MSSubClass_80',
 'MSSubClass_85',
 'MSSubClass_90']

In [121]:
OTHER_PREDICTORS = list(['LogGrLivArea', 'OverallQual', 'OverallCond', 'GarageArea'])



In [122]:
for elem in MSSUBCLASS_ONE_HOT_COLS:
    OTHER_PREDICTORS.append(elem)

In [123]:
for elem in NEIGHBORHOOD_ONE_HOT_COLS:
    OTHER_PREDICTORS.append(elem)

In [124]:
for elem in BSMT_QUAL_ONE_HOT_COLS:
    OTHER_PREDICTORS.append(elem)
    

In [125]:
OTHER_PREDICTORS

['LogGrLivArea',
 'OverallQual',
 'OverallCond',
 'GarageArea',
 'MSSubClass_120',
 'MSSubClass_150',
 'MSSubClass_160',
 'MSSubClass_180',
 'MSSubClass_190',
 'MSSubClass_20',
 'MSSubClass_30',
 'MSSubClass_40',
 'MSSubClass_45',
 'MSSubClass_50',
 'MSSubClass_60',
 'MSSubClass_70',
 'MSSubClass_75',
 'MSSubClass_80',
 'MSSubClass_85',
 'MSSubClass_90',
 'Neighborhood_Blmngtn',
 'Neighborhood_Blueste',
 'Neighborhood_BrDale',
 'Neighborhood_BrkSide',
 'Neighborhood_ClearCr',
 'Neighborhood_CollgCr',
 'Neighborhood_Crawfor',
 'Neighborhood_Edwards',
 'Neighborhood_Gilbert',
 'Neighborhood_IDOTRR',
 'Neighborhood_MeadowV',
 'Neighborhood_Mitchel',
 'Neighborhood_NAmes',
 'Neighborhood_NPkVill',
 'Neighborhood_NWAmes',
 'Neighborhood_NoRidge',
 'Neighborhood_NridgHt',
 'Neighborhood_OldTown',
 'Neighborhood_SWISU',
 'Neighborhood_Sawyer',
 'Neighborhood_SawyerW',
 'Neighborhood_Somerst',
 'Neighborhood_StoneBr',
 'Neighborhood_Timber',
 'Neighborhood_Veenker',
 'BsmtQual_Ex',
 'BsmtQual_

In [126]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot_new.sample(frac=1),  
    X_columns=OTHER_PREDICTORS,
    X_column_transform_map={'MSSubClass' : get_Group_LogSalePricePerSqFeet_transformation,
                            'YrSold' : get_Group_LogSalePricePerSqFeet_transformation,
                            'Functional' : get_Group_LogSalePricePerSqFeet_transformation,                            
                            'KitchenQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageQual' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'GarageFinish' : get_Group_LogSalePricePerSqFeet_transformation,                                                        
                            'Neighborhood' : get_Group_LogSalePrice_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())
 

[0.13357754 0.12919867 0.14428084 0.16266166 0.14082878]
0.1421094986061739
0.01156053798088997


This brings us back to square one. It looks like the one hot encoding, after all is not such a bad way to numerically represent categorical variables and is performing better than our scheme using the average log sales price.

Let us design a transform function for doing one hot encodings easily.

In [127]:
def get_Group_one_hot_encoded_transformation(group_col, train_data, validation_data):
    
    rel_cols = [col for col in train_data.columns if group_col + '_' in col and group_col + '_Val' not in col]
    return rel_cols


In [128]:
get_Group_one_hot_encoded_transformation('MSSubClass', train_validation_data_one_hot, validation_data_one_hot)

['MSSubClass_120',
 'MSSubClass_150',
 'MSSubClass_160',
 'MSSubClass_180',
 'MSSubClass_190',
 'MSSubClass_20',
 'MSSubClass_30',
 'MSSubClass_40',
 'MSSubClass_45',
 'MSSubClass_50',
 'MSSubClass_60',
 'MSSubClass_70',
 'MSSubClass_75',
 'MSSubClass_80',
 'MSSubClass_85',
 'MSSubClass_90']

In [129]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot_new.sample(frac=1),  
    X_columns=['LogGrLivArea', 'MSSubClass', 'OverallQual', 'OverallCond', 
               'Neighborhood', 'GarageArea', 'BsmtQual'],
    X_column_transform_map={'MSSubClass' : get_Group_one_hot_encoded_transformation,
                            'BsmtQual' : get_Group_one_hot_encoded_transformation,                            
                            'Neighborhood' : get_Group_one_hot_encoded_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())
 

[0.13438854 0.15129898 0.15145654 0.13758468 0.14392466]
0.1437306802791411
0.0069577970401299


#### Won't linear regression blow up if we have linearly dependent predictors.

This is a reasonable question, and though it does not appear to be doing so, let us confirm the same.

In [130]:
X1 = np.random.rand(100)
X2 = np.random.rand(100)
X3 = 1.0 - X1 - X2
Y = 2*X1 + 5*X2 + 0.005*np.random.rand(100)
X = np.zeros((100,3))
X[:,0] = X1
X[:,1] = X2
X[:,2] = X3

 

In [131]:
np.shape(Y)

(100,)

In [132]:
np.shape(X[:,0:3])

(100, 3)

In [133]:
my_model = linear_model.LinearRegression()
my_model.fit(X[:,0:2],Y)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [134]:
my_model.intercept_

0.0030941075193511303

In [135]:
my_model.coef_

array([1.99993046, 4.99933721])

In [136]:
my_model.score(X[:,0:2],Y)

0.9999991504339908

As expected the model does not blow up. However we can see that the coefficients would end up with some being positive and some being negative to neutralize the effect. Since the essential model remains unchanged, let us ignore this for now.

Let us just try adding predictors and making some models.

In [146]:
cross_validation_scores = get_cross_val_output( 
    train_validation_data_one_hot_new.sample(frac=1),  
    X_columns=['LogGrLivArea', 
               'MSZoning',
               'Utilities',
               'LotConfig',
               'MSSubClass', 
               'Neighborhood', 
               'OverallQual', 
               'OverallCond', 
               'GarageArea', 
               'BsmtQual',
               'KitchenQual'],
    X_column_transform_map={'MSSubClass' : get_Group_one_hot_encoded_transformation,
                            'MSZoning' : get_Group_one_hot_encoded_transformation,                            
                            'LotConfig' : get_Group_one_hot_encoded_transformation,                                                        
                            'Utilities' : get_Group_one_hot_encoded_transformation,                                                        
                            'LandSlope' : get_Group_one_hot_encoded_transformation,                                                        
                            'BsmtQual' : get_Group_one_hot_encoded_transformation,                            
                            'KitchenQual' : get_Group_one_hot_encoded_transformation,                                                        
                            'Neighborhood' : get_Group_one_hot_encoded_transformation})
print(cross_validation_scores)
print(cross_validation_scores.mean())
print(cross_validation_scores.std())
  

[1.58404681e-01 1.28288666e-01 4.02710942e+05 1.29747114e-01
 7.05131359e+04]
94644.89888424877
156435.23947865062


### How to go about and when to stop ?
We can  go about adding new predictors and they might decrease the cross validation score. But how do we finalize and where do we stop ?

It makes sense to use a modeling techinique called lasso, which would take of this increased dimensionality problem and give us an optimal model with fewer dimensions.

Though, we have not used this technique in the current iteration, we had used it before and now hopefully , with our cleaned up and transformed data, it should yield better results.